In [1]:
import syft as sy
import os

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
# node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
#                            reset=True,
#                            n_consumers=4,
#                            create_producer=True)
# client = node.login(email="info@openmined.org", password="changethis")
client = sy.login(url="http://localhost:8080", email="info@openmined.org", password="changethis")

Logged into <xenodochial_lecun: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:

# from syft.store.blob_storage import BlobStorageConfig, BlobStorageClientConfig
# from syft.store.blob_storage.seaweedfs import SeaweedFSClient, SeaweedFSClientConfig
# blob_config = BlobStorageConfig(client_type=SeaweedFSClient,
#                                 client_config=SeaweedFSClientConfig(host="http://0.0.0.0",
#                                                                     port="8333",
#                                                                     access_key="admin",
#                                                                     secret_key="admin",
#                                                                     bucket_name="test_bucket",
#                                                                     region="us-east-1", 
#                                                                     # mount_port=4001
#                                                                    )
# )
# node.python_node.init_blob_storage(blob_config)

In [4]:
client.api.services.blob_storage.mount_azure(
    account_name='helmprojectstorage',
    container_name='helm',
    account_key=os.environ["HELM_STORAGE_ACCOUNT_KEY"],
    bucket_name='helmazurebucket',
)

SyftSuccess: Mounting Azure Successful!

In [5]:
files = client.api.services.blob_storage.get_files_from_bucket("helmazurebucket")

In [6]:
files

[syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile]

In [10]:
file = [f for f in files if f.file_name=="test.json"][0]

In [11]:
file

```python
class BlobFile:
  id: str = bf18dfa66d144e828cf6af41078233ad
  file_name: str = "test.json"

```

In [12]:
file.read()

b'{\n"abc": "def"\n}'